In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/The-Role-of-Spatial-Context-in-Deep-Learning-based-Semantic-Segmentation-of-Remote-Sensing-Imagery/dfc20/
!pip install rasterio

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import os
import sys
from torchvision import transforms


project_root = os.getcwd()
sys.path.append(os.path.join(project_root, "models"))
from dataset import *
from models.unet import UNet

In [ ]:
path = "./data"

# load datasets
train_set = DFC20(path,
                  subset="train",
                  use_s1=False,
                  use_s2_RGB=True,
                  use_s2_hr=False,
                  use_s2_all=False,
                  as_tensor=True)

val_set = DFC20(path,
                subset="val",
                use_s1=False,
                use_s2_RGB=True,
                use_s2_hr=False,
                use_s2_all=False,
                as_tensor=True)

n_inputs = train_set.n_inputs

# set up dataloaders
train_loader = DataLoader(train_set,
                            batch_size=8,
                            shuffle=True,
                            pin_memory=True,
                            drop_last=False)

val_loader = DataLoader(train_set,
                            batch_size=8,
                            shuffle=True,
                            pin_memory=True,
                            drop_last=False)

model = UNet(n_channels=n_inputs)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = model.to(device)

[Load]: 100%|██████████| 684/684 [00:00<00:00, 85518.94it/s]

loaded 684 samples from the DFC20 subset val


In [3]:
train_set.__getitem__(0)['label']

tensor([[6, 6, 6,  ..., 6, 6, 6],
        [6, 6, 6,  ..., 6, 6, 6],
        [6, 6, 6,  ..., 6, 6, 6],
        ...,
        [1, 1, 1,  ..., 4, 6, 6],
        [1, 1, 1,  ..., 4, 6, 6],
        [1, 1, 1,  ..., 4, 6, 6]])

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True
)

# mIoU helper
def compute_miou(pred, label, num_classes):
    ious = []
    pred = pred.view(-1)
    label = label.view(-1)
    for cls in range(num_classes):
        pred_inds = (pred == cls)
        label_inds = (label == cls)
        intersection = (pred_inds & label_inds).sum().item()
        union = (pred_inds | label_inds).sum().item()
        if union == 0:
            ious.append(float('nan'))  # or 0.0
        else:
            ious.append(intersection / union)
    return sum([iou for iou in ious if not torch.isnan(torch.tensor(iou))]) / num_classes

# Training loop
writer = SummaryWriter()
best_val_accuracy = 0.0
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_accuracy = 0.0
    total_miou = 0.0
    num_batches = 0

    for batch in train_loader:
        inputs = batch['image'].to(device)
        labels = batch['label'].to(device) - 1

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        # Optional: Gradient Clipping
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        running_loss += loss.item()

        predicted_labels = torch.argmax(outputs, dim=1)
        accuracy = (predicted_labels == labels).float().mean().item() * 100
        miou = compute_miou(predicted_labels, labels, num_classes=outputs.shape[1])

        total_accuracy += accuracy
        total_miou += miou
        num_batches += 1

    avg_loss = running_loss / len(train_loader)
    avg_accuracy = total_accuracy / num_batches
    avg_miou = total_miou / num_batches

    # Validation step
    model.eval()
    val_loss = 0.0
    val_accuracy = 0.0
    val_miou = 0.0
    with torch.no_grad():
        for batch in val_loader:
            inputs = batch['image'].to(device)
            labels = batch['label'].to(device) - 1

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            predicted = torch.argmax(outputs, dim=1)
            acc = (predicted == labels).float().mean().item() * 100
            miou = compute_miou(predicted, labels, num_classes=outputs.shape[1])

            val_accuracy += acc
            val_miou += miou

    val_loss /= len(val_loader)
    val_accuracy /= len(val_loader)
    val_miou /= len(val_loader)

    # Step the LR scheduler based on val loss
    scheduler.step(val_loss)

    # Print metrics
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"  Train     — Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.2f}%, mIoU: {avg_miou:.4f}")
    print(f"  Validate  — Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%, mIoU: {val_miou:.4f}")

    # TensorBoard logging
    writer.add_scalar('Loss/train', avg_loss, epoch)
    writer.add_scalar('Accuracy/train', avg_accuracy, epoch)
    writer.add_scalar('mIoU/train', avg_miou, epoch)

    writer.add_scalar('Loss/val', val_loss, epoch)
    writer.add_scalar('Accuracy/val', val_accuracy, epoch)
    writer.add_scalar('mIoU/val', val_miou, epoch)
    writer.add_scalar('LearningRate', optimizer.param_groups[0]['lr'], epoch)

    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), 'trained_models/unet_baseline.pth') # change this for diff models

writer.close()


IndexError: Target 10 is out of bounds.